# prepare tensorboard log

generate code: gen_log.py
```gen_log.py
from torch.utils.tensorboard import SummaryWriter


for i in range(5):
    save_metrics = {'train/acc': None, 'train/loss': None}
    writer = SummaryWriter(f'runs/{i}')
    for step in range(50):
        writer.add_scalar('train/acc', 10*i+step, step)
        writer.add_scalar('train/loss', 10*i - step, step)
    writer.add_hparams({'lr': 0.1*i, 'bsize': i, 'gpu': True, 'opt': 'adam'}, save_metrics)
writer.close()
```
output file structure

```
runs
├── 0
│   └── events.out.tfevents.xxx.0
├── 1
│   └── events.out.tfevents..xxx.1
├── 2
│   └── events.out.tfevents.xxx.2
├── 3
│   └── events.out.tfevents.xxx.3
└── 4
    └── events.out.tfevents.xxx.4
```

In [1]:
#!rm -rf runs # remove log
#!python gen_log.py  # generate log

In [2]:
%load_ext tensorboard
%tensorboard  --logdir runs

# Convert tensorboard log to pandas

In [3]:
from tensorboardpy import logdir
from tensorboardpy import select_steps

In [4]:
df = logdir('runs', ignore_path=True)

events files loading...
[   'runs/0/events.out.tfevents.1629596494.yamada.local.43744.0',
    'runs/1/events.out.tfevents.1629596494.yamada.local.43744.1',
    'runs/4/events.out.tfevents.1629596494.yamada.local.43744.4',
    'runs/3/events.out.tfevents.1629596494.yamada.local.43744.3',
    'runs/2/events.out.tfevents.1629596494.yamada.local.43744.2']


In [5]:
df

,metric,value,step,bsize,opt,gpu,lr
0,train/acc,0.0,0.0,0.0,adam,1.0,0.0
1,train/acc,1.0,1.0,0.0,adam,1.0,0.0
2,train/acc,2.0,2.0,0.0,adam,1.0,0.0
3,train/acc,3.0,3.0,0.0,adam,1.0,0.0
4,train/acc,4.0,4.0,0.0,adam,1.0,0.0
...,...,...,...,...,...,...,...
495,train/loss,-25.0,45.0,2.0,adam,1.0,0.2
496,train/loss,-26.0,46.0,2.0,adam,1.0,0.2
497,train/loss,-27.0,47.0,2.0,adam,1.0,0.2
498,train/loss,-28.0,48.0,2.0,adam,1.0,0.2


# Useful statistical operations for writing papers

In [6]:
# shrink the steps according to the rules
ret_df = select_steps(df, metric='train/acc', rule='max')
ret_df

,opt,lr,gpu,step,bsize,train/acc,train/loss
0,adam,0.0,1.0,49.0,0.0,49.0,-49.0
1,adam,0.1,1.0,49.0,1.0,59.0,-39.0
2,adam,0.2,1.0,49.0,2.0,69.0,-29.0
3,adam,0.3,1.0,49.0,3.0,79.0,-19.0
4,adam,0.4,1.0,49.0,4.0,89.0,-9.0
